In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col
spark = SparkSession.builder.appName("Jupyter").getOrCreate() #Spark is a JVM lib so we have camel case instead of snake case

events = spark.read.option(
    "header", "true"
).csv(
    "/home/iceberg/data/events.csv"
).withColumn(
    "event_date", expr("DATE_TRUNC('day', event_time)")
)



25/07/10 13:35:01 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| device_id|    user_id|referrer|                host|       url|          event_time|         event_date|browser_family|os_family|device_type|
+----------+-----------+--------+--------------------+----------+--------------------+-------------------+--------------+---------+-----------+
| 532630305| 1037710827|    NULL| www.zachwilson.tech|         /|2021-03-08 17:27:...|2021-03-08 00:00:00|         Other|    Other|      Other|
| 532630305|  925588856|    NULL|    www.eczachly.com|         /|2021-05-10 11:26:...|2021-05-10 00:00:00|         Other|    Other|      Other|
| 532630305|-1180485268|    NULL|admin.zachwilson....|         /|2021-02-17 16:19:...|2021-02-17 00:00:00|         Other|    Other|      Other|
| 532630305|-1044833855|    NULL| www.zachwilson.tech|         /|2021-09-24 15:53:...|2021-09-24 00:00:00|         Other|    Other|     

In [3]:
# events.collect() Don't put the entire dataset on the driver my bro

In [ ]:
devices = spark.read.option("header","true").csv("/home/iceberg/data/devices.csv")

df = events.join(devices,on="device_id",how="left")
df = df.withColumnsRenamed({'browser_type': 'browser_family', 'os_type': 'os_family'})

df.show()

In [9]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.explain()
sortedTwo.explain()


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#165, event_date#29, browser_family#74, os_family#75, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=210]
         +- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56 AS browser_family#74, os_type#57 AS os_family#75, device_type#58]
            +- BroadcastHashJoin [device_id#18], [device_id#55], LeftOuter, BuildRight, false
               :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               :  +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters:

== Physical Plan ==                          ONLY 1 SHUFFLE HERE (BECAUSE WE HAVE THE REPARTITION())
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#165, event_date#29, browser_family#74, os_family#75, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=210]
         +- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56 AS browser_family#74, os_type#57 AS os_family#75, device_type#58]
            +- BroadcastHashJoin [device_id#18], [device_id#55], LeftOuter, BuildRight, false
               :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
               :  +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:string,device_id:string,referrer:string,host:string,url:string,event_time:string>
               +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=207]
                  +- Filter isnotnull(device_id#55)
                     +- FileScan csv [device_id#55,browser_type#56,os_type#57,device_type#58] Batched: false, DataFilters: [isnotnull(device_id#55)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/devices.csv], PartitionFilters: [], PushedFilters: [IsNotNull(device_id)], ReadSchema: struct<device_id:string,browser_type:string,os_type:string,device_type:string>


== Physical Plan == EXCHANGE MEANS SHUFFLE            HERE WE 2X SHUFFLE
AdaptiveSparkPlan isFinalPlan=false
+- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#176, event_date#29, browser_family#74, os_family#75, device_type#58]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], true, 0
      +- Exchange rangepartitioning(event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST, 200), ENSURE_REQUIREMENTS, [plan_id=257] ¡¡PAINFUL!!
         +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=255]
            +- Project [device_id#18, user_id#17, referrer#19, host#20, url#21, event_time#22, event_date#29, browser_type#56 AS browser_family#74, os_type#57 AS os_family#75, device_type#58]
               +- BroadcastHashJoin [device_id#18], [device_id#55], LeftOuter, BuildRight, false
                  :- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
                  :  +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:string,device_id:string,referrer:string,host:string,url:string,event_time:string>
                  +- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [plan_id=252]
                     +- Filter isnotnull(device_id#55)
                        +- FileScan csv [device_id#55,browser_type#56,os_type#57,device_type#58] Batched: false, DataFilters: [isnotnull(device_id#55)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/devices.csv], PartitionFilters: [], PushedFilters: [IsNotNull(device_id)], ReadSchema: struct<device_id:string,browser_type:string,os_type:string,device_type:string>

In [ ]:
# .sortWithinPartitions() sorts within partitions, whereas .sort() is a global sort, which is very slow

# Note - exchange is synonymous with Shuffle

In [6]:
sorted = df.repartition(10, col("event_date"))\
    .sortWithinPartitions(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sortedTwo = df.repartition(10, col("event_date"))\
    .sort(col("event_date"), col("host"))\
    .withColumn("event_time", col("event_time").cast("timestamp")) 

sorted.explain()
sortedTwo.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, cast(event_time#22 as timestamp) AS event_time#288, event_date#29]
   +- Sort [event_date#29 ASC NULLS FIRST, host#20 ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(event_date#29, 10), REPARTITION_BY_NUM, [plan_id=294]
         +- Project [user_id#17, device_id#18, referrer#19, host#20, url#21, event_time#22, date_trunc(day, cast(event_time#22 as timestamp), Some(Etc/UTC)) AS event_date#29]
            +- FileScan csv [user_id#17,device_id#18,referrer#19,host#20,url#21,event_time#22] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/iceberg/data/events.csv], PartitionFilters: [], PushedFilters: [], ReadSchema: struct<user_id:string,device_id:string,referrer:string,host:string,url:string,event_time:string>


== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Project [user_id#17, device_id#18, referr

In [10]:
%%sql

CREATE DATABASE IF NOT EXISTS bootcamp

++
||
++
++

In [11]:
%%sql

DROP TABLE IF EXISTS bootcamp.events

++
||
++
++

In [12]:
%%sql

DROP TABLE IF EXISTS bootcamp.events_sorted

++
||
++
++

In [13]:
%%sql

CREATE TABLE IF NOT EXISTS bootcamp.events (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));


++
||
++
++

In [14]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_sorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (years(event_date));

++
||
++
++

In [24]:
%%sql


CREATE TABLE IF NOT EXISTS bootcamp.events_unsorted (
    url STRING,
    referrer STRING,
    browser_family STRING,
    os_family STRING,
    device_family STRING,
    host STRING,
    event_time TIMESTAMP,
    event_date DATE
)
USING iceberg
PARTITIONED BY (year(event_date));

++
||
++
++

In [21]:

start_df = df.repartition(4, col("event_date")).withColumn("event_time", col("event_time").cast("timestamp")) \
    
first_sort_df = start_df.sortWithinPartitions(col("event_date"), col("browser_family"), col("host"))
                                            # See this! Lowest to highest cardinality
start_df.write.mode("overwrite").saveAsTable("bootcamp.events_unsorted")
first_sort_df.write.mode("overwrite").saveAsTable("bootcamp.events_sorted")

In [22]:
%%sql

SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'sorted' 
FROM demo.bootcamp.events_sorted.files

UNION ALL
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files, 'unsorted' 
FROM demo.bootcamp.events_unsorted.files

--You will get smaller size


size,num_files,sorted
5517729,4,sorted
5553006,4,unsorted


In [25]:
%%sql
SELECT * FROM demo.bootcamp.events_sorted.files

content,file_path,file_format,spec_id,partition,record_count,file_size_in_bytes,column_sizes,value_counts,null_value_counts,nan_value_counts,lower_bounds,upper_bounds,key_metadata,split_offsets,equality_ids,sort_order_id,readable_metrics
0,s3://warehouse/bootcamp/events_sorted/data/00000-95-e2202785-abd0-49e6-bfcb-a047a4709422-0-00001.parquet,PARQUET,1,Row(event_date_year=None),89391,1115080,"{1: 113861, 2: 70961, 3: 48357, 4: 26364, 6: 16032, 7: 379126, 8: 2274, 9: 105264, 10: 316924, 11: 31485}","{1: 89391, 2: 89391, 3: 89391, 4: 89391, 6: 89391, 7: 89391, 8: 89391, 9: 89391, 10: 89391, 11: 89391}","{1: 0, 2: 46359, 3: 0, 4: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 1, 11: 0}",{},"{1: bytearray(b'/'), 2: bytearray(b'52.20.78.240'), 3: bytearray(b'%E3%82%A6%E3%82%'), 4: bytearray(b'Android'), 6: bytearray(b'aashish.techcrea'), 7: bytearray(b' \xba\xe7\xb8\xa8\xb8\x05\x00'), 8: bytearray(b'\x00\xa0&\xb4\xa8\xb8\x05\x00'), 9: bytearray(b'-100210680'), 10: bytearray(b'-1000095488'), 11: bytearray(b'17MB150WB')}","{1: bytearray(b'/zzageqnf.php?Fp'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 6: bytearray(b'zachwilson.techd'), 7: bytearray(b'\xe8\xb0\x1b\x8ec\x03\x06\x00'), 8: bytearray(b'\x00\xe0dqO\x03\x06\x00'), 9: bytearray(b'999535123'), 10: bytearray(b'999884938'), 11: bytearray(b'vivo $2')}",None,[4],None,0,"Row(browser_family=Row(column_size=48357, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='%E3%82%A6%E3%82%', upper_bound='webprosbot'), device_id=Row(column_size=105264, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='-100210680', upper_bound='999535123'), device_type=Row(column_size=31485, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='17MB150WB', upper_bound='vivo $2'), event_date=Row(column_size=2274, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 0), upper_bound=datetime.datetime(2023, 8, 20, 0, 0)), event_time=Row(column_size=379126, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound=datetime.datetime(2021, 1, 12, 0, 1, 19, 764000), upper_bound=datetime.datetime(2023, 8, 20, 23, 59, 41, 89000)), host=Row(column_size=16032, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='aashish.techcrea', upper_bound='zachwilson.techd'), os_family=Row(column_size=26364, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='Android', upper_bound='iOS'), referrer=Row(column_size=70961, value_count=89391, null_value_count=46359, nan_value_count=None, lower_bound='52.20.78.240', upper_bound='zachwilson.tech'), url=Row(column_size=113861, value_count=89391, null_value_count=0, nan_value_count=None, lower_bound='/', upper_bound='/zzageqnf.php?Fp'), user_id=Row(column_size=316924, value_count=89391, null_value_count=1, nan_value_count=None, lower_bound='-1000095488', upper_bound='999884938'))"
0,s3://warehouse/bootcamp/events_sorted/data/00001-96-e2202785-abd0-49e6-bfcb-a047a4709422-0-00001.parquet,PARQUET,1,Row(event_date_year=None),99232,1257769,"{1: 145108, 2: 76067, 3: 49434, 4: 35407, 6: 20841, 7: 422816, 8: 2355, 9: 119598, 10: 346010, 11: 35662}","{1: 99232, 2: 99232, 3: 99232, 4: 99232, 6: 99232, 7: 99232, 8: 99232, 9: 99232, 10: 99232, 11: 99232}","{1: 0, 2: 49299, 3: 0, 4: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 58, 11: 0}",{},"{1: bytearray(b'""/?""""<?=print(93'), 2: bytearray(b'""https://www.goo'), 3: bytearray(b') Bot'), 4: bytearray(b'Android'), 6: bytearray(b'abhishekanand.te'), 7: bytearray(b'(\x83\xb2EX\xb8\x05\x00'), 8: bytearray(b'\x00 \xc9<X\xb8\x05\x00'), 9: bytearray(b'-100210680'), 10: bytearray(b'-1000370060'), 11: bytearray(b'13 Pro Max')}","{1: bytearray(b'/zz.php'), 2: bytearray(b'zachwilson.tech'), 3: bytearray(b'webprosbot'), 4: bytearray(b'iOS'), 6: bytearray(b'zsavi524.techcrf'), 7: bytearray(b'\x88\xb8\x07P;\x03\x06\x00'), 8: bytearray(b""\x00 \xb65\'\x03\x06\x00""), 9: bytearray(b

In [ ]:
# If you sort u will get better partitions

In [17]:
%%sql
SELECT SUM(file_size_in_bytes) as size, COUNT(1) as num_files FROM demo.bootcamp.events.files;

size,num_files
None,0


In [18]:
%%sql 
SELECT COUNT(1) FROM bootcamp.matches_bucketed.files

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `bootcamp`.`matches_bucketed`.`files` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 1 pos 21;
'Aggregate [unresolvedalias(count(1), None)]
+- 'UnresolvedRelation [bootcamp, matches_bucketed, files], [], false


count(1)
3665
